In [72]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

%matplotlib inline

In [73]:
# Объявим функцию, которая позволит проверять гипотезы с помощью бутстрапа (mean)
def get_bootstrap_mean(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.mean, # интересующая нас статистика
    bootstrap_conf_level = 0.95 # уровень значимости
):
    boot_len = max([len(data_column_1), len(data_column_2)])
    boot_data = []
    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            boot_len, 
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            boot_len, 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1-samples_2)) # mean() - применяем статистику
        
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    ci = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
        
    # Визуализация
    plt.hist(pd_boot_data[0], bins = 50)
    
    plt.style.use('ggplot')
    plt.vlines(ci,ymin=0,ymax=50,linestyle='--')
    plt.xlabel('boot_data')
    plt.ylabel('frequency')
    plt.title("Histogram of boot_data")
    plt.show()
       
    return {"boot_data": boot_data, 
            "ci": ci, 
            "p_value": p_value}

In [74]:
# Объявим функцию, которая позволит проверять гипотезы с помощью бутстрапа (median)
def get_bootstrap_median(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.median, # интересующая нас статистика
    bootstrap_conf_level = 0.95 # уровень значимости
):
    boot_len = max([len(data_column_1), len(data_column_2)])
    boot_data = []
    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            boot_len, 
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            boot_len, 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1-samples_2)) # mean() - применяем статистику
        
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    ci = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
        
    # Визуализация
    plt.hist(pd_boot_data[0], bins = 50)
    
    plt.style.use('ggplot')
    plt.vlines(ci,ymin=0,ymax=50,linestyle='--')
    plt.xlabel('boot_data')
    plt.ylabel('frequency')
    plt.title("Histogram of boot_data")
    plt.show()
       
    return {"boot_data": boot_data, 
            "ci": ci, 
            "p_value": p_value}

# Задача:

#### 1. Сравните результат между тестом и контролем по двум кейсам:
    Примените бутстрап (с np.mean) и критерий mann-whitney, а потом сравните p-value.
    Примените бутстрап (с np.median) и критерий mann-whitney, а потом сравните p-value.
#### 2. Напишите выводы, которые можно сделать на основе анализа примененных критериев

In [60]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/396008/hw_bootstrap.csv', sep=';')

#### Описание колонок:
    1. value – значения метрики
    2. experimentVariant – Вариант теста (Control – контроль, Treatment – тест)

In [61]:
df.head()

,Unnamed: 0,value,experimentVariant
0,1,"10,3804953740268",Control
1,2,"9,54686666784264",Control
2,3,"11,0882146509904",Control
3,4,"10,1472740570122",Control
4,5,"9,78980767524929",Control


In [62]:
df.tail()

,Unnamed: 0,value,experimentVariant
995,996,1000,Treatment
996,997,1200,Treatment
997,998,1500,Treatment
998,999,2000,Treatment
999,1000,3000,Treatment


In [63]:
print(f'Количесвто строк: {df.shape[0]}, количество колонок: {df.shape[1]}')

Количесвто строк: 1000, количество колонок: 3


In [64]:
df = df \
    .drop(columns=('Unnamed: 0')) \
    .rename(columns=({'experimentVariant':'experiment_variant'}))

In [65]:
df.head()

,value,experiment_variant
0,"10,3804953740268",Control
1,"9,54686666784264",Control
2,"11,0882146509904",Control
3,"10,1472740570122",Control
4,"9,78980767524929",Control


In [66]:
df.experiment_variant.value_counts()

Treatment    500
Control      500
Name: experiment_variant, dtype: int64

In [67]:
df.value.value_counts()

12,2354526719882    2
11,2868402712262    2
10,6835309806202    2
11,5917945205843    2
8,35554786067164    2
                   ..
11,1690784219038    1
1500                1
1000                1
10,9739287498498    1
11,4995664874117    1
Name: value, Length: 505, dtype: int64

In [68]:
# Видим, что колонка value представлена в виде строки, хотя внутри значения численные.
# Также, целая и дроьная часть разделены запятой, а не точкой, что не позволит нам изменить формат на float.
df.dtypes

value                 object
experiment_variant    object
dtype: object

In [69]:
df.value = df.value.str.replace(',', '.').astype(float)

In [75]:
df.head()

,value,experiment_variant
0,10.380495,Control
1,9.546867,Control
2,11.088215,Control
3,10.147274,Control
4,9.789808,Control


In [82]:
treatment_df = df.query('experiment_variant == "Treatment"').reset_index(drop='yes')
treatment_df.head()

,value,experiment_variant
0,10.380495,Treatment
1,9.546867,Treatment
2,11.088215,Treatment
3,10.147274,Treatment
4,9.789808,Treatment


In [80]:
control_df = df.query('experiment_variant == "Control"')
control_df.head()

,value,experiment_variant
0,10.380495,Control
1,9.546867,Control
2,11.088215,Control
3,10.147274,Control
4,9.789808,Control
